### ENVIRONMENT

In [1]:
"""
This is a gym environment based for the drone_game
The goal is to reach randomly positoned targets
"""

import os
from math import sin, cos, pi, sqrt
from random import randrange

import numpy as np
import gym
from gym import spaces

import pygame
from pygame.locals import *


class droneEnv(gym.Env):
    def __init__(self, render_every_frame, mouse_target):
        super(droneEnv, self).__init__()

        self.render_every_frame = render_every_frame
        # Makes the target follow the mouse
        self.mouse_target = mouse_target

        # Initialize Pygame, load sprites
        pygame.init()
        self.screen = pygame.display.set_mode((800, 800))
        self.FramePerSec = pygame.time.Clock()

        self.player = pygame.image.load(os.path.join("assets/sprites/drone_old.png"))
        self.player.convert()

        self.target = pygame.image.load(os.path.join("assets/sprites/target_old.png"))
        self.target.convert()

        pygame.font.init()
        self.myfont = pygame.font.SysFont("Comic Sans MS", 20)

        # Physics constants
        self.FPS = 60
        self.gravity = 0.08
        self.thruster_amplitude = 0.04
        self.diff_amplitude = 0.003
        self.thruster_mean = 0.04
        self.mass = 1
        self.arm = 25

        # Initialize variables
        (self.a, self.ad, self.add) = (0, 0, 0)
        (self.x, self.xd, self.xdd) = (400, 0, 0)
        (self.y, self.yd, self.ydd) = (400, 0, 0)
        self.xt = randrange(200, 600)
        self.yt = randrange(200, 600)

        # Initialize game variables
        self.target_counter = 0
        self.reward = 0
        self.time = 0
        self.time_limit = 20
        if self.mouse_target is True:
            self.time_limit = 1000

        # 2 action thrust amplitude and thrust difference in float values between -1 and 1
        self.action_space = spaces.Box(low=-1, high=1, shape=(2,))
        # 6 observations: angle_to_up, velocity, angle_velocity, distance_to_target, angle_to_target, angle_target_and_velocity
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(6,))

    def reset(self):
        # Reset variables
        (self.a, self.ad, self.add) = (0, 0, 0)
        (self.x, self.xd, self.xdd) = (400, 0, 0)
        (self.y, self.yd, self.ydd) = (400, 0, 0)
        self.xt = randrange(200, 600)
        self.yt = randrange(200, 600)

        self.target_counter = 0
        self.reward = 0
        self.time = 0

        return self.get_obs()

    def get_obs(self) -> np.ndarray:
        """
        Calculates the observations

        Returns:
            np.ndarray: The normalized observations:
            - angle_to_up : angle between the drone and the up vector (to observe gravity)
            - velocity : velocity of the drone
            - angle_velocity : angle of the velocity vector
            - distance_to_target : distance to the target
            - angle_to_target : angle between the drone and the target
            - angle_target_and_velocity : angle between the to_target vector and the velocity vector
        """
        angle_to_up = self.a / 180 * pi
        velocity = sqrt(self.xd**2 + self.yd**2)
        angle_velocity = self.ad
        distance_to_target = (
            sqrt((self.xt - self.x) ** 2 + (self.yt - self.y) ** 2) / 500
        )
        angle_to_target = np.arctan2(self.yt - self.y, self.xt - self.x)
        # Angle between the to_target vector and the velocity vector
        angle_target_and_velocity = np.arctan2(
            self.yt - self.y, self.xt - self.x
        ) - np.arctan2(self.yd, self.xd)
        distance_to_target = (
            sqrt((self.xt - self.x) ** 2 + (self.yt - self.y) ** 2) / 500
        )
        return np.array(
            [
                angle_to_up,
                velocity,
                angle_velocity,
                distance_to_target,
                angle_to_target,
                angle_target_and_velocity,
            ]
        ).astype(np.float32)

    def step(self, action):
        # Game loop
        self.reward = 0.0
        (action0, action1) = (action[0], action[1])

        # Act every 5 frames
        for _ in range(5):
            self.time += 1 / 60

            if self.mouse_target is True:
                self.xt, self.yt = pygame.mouse.get_pos()

            # Initialize accelerations
            self.xdd = 0
            self.ydd = self.gravity
            self.add = 0
            thruster_left = self.thruster_mean
            thruster_right = self.thruster_mean

            thruster_left += action0 * self.thruster_amplitude
            thruster_right += action0 * self.thruster_amplitude
            thruster_left += action1 * self.diff_amplitude
            thruster_right -= action1 * self.diff_amplitude

            # Calculating accelerations with Newton's laws of motions
            self.xdd += (
                -(thruster_left + thruster_right) * sin(self.a * pi / 180) / self.mass
            )
            self.ydd += (
                -(thruster_left + thruster_right) * cos(self.a * pi / 180) / self.mass
            )
            self.add += self.arm * (thruster_right - thruster_left) / self.mass

            self.xd += self.xdd
            self.yd += self.ydd
            self.ad += self.add
            self.x += self.xd
            self.y += self.yd
            self.a += self.ad

            dist = sqrt((self.x - self.xt) ** 2 + (self.y - self.yt) ** 2)

            # Reward per step survived
            self.reward += 1 / 60
            # Penalty according to the distance to target
            self.reward -= dist / (100 * 60)

            if dist < 50:
                # Reward if close to target
                self.xt = randrange(200, 600)
                self.yt = randrange(200, 600)
                self.reward += 100
                
                
                # 
                self.target_counter+=1

            # If out of time
            if self.time > self.time_limit:
                done = True
                break

            # If too far from target (crash)
            elif dist > 1000:
                self.reward -= 1000
                done = True
                break

            else:
                done = False

            if self.render_every_frame is True:
                self.render("yes")

        info = {}

        return (
            self.get_obs(),
            self.reward,
            done,
            info,
        )

    def render(self, mode):
        # Pygame rendering
        pygame.event.get()
        self.screen.fill(0)
        self.screen.blit(
            self.target,
            (
                self.xt - int(self.target.get_width() / 2),
                self.yt - int(self.target.get_height() / 2),
            ),
        )
        player_copy = pygame.transform.rotate(self.player, self.a)
        self.screen.blit(
            player_copy,
            (
                self.x - int(player_copy.get_width() / 2),
                self.y - int(player_copy.get_height() / 2),
            ),
        )

        textsurface = self.myfont.render(
            "Collected: " + str(self.target_counter), False, (255, 255, 255)
        )
        self.screen.blit(textsurface, (20, 20))
        textsurface3 = self.myfont.render(
            "Time: " + str(int(self.time)), False, (255, 255, 255)
        )
        self.screen.blit(textsurface3, (20, 50))

        pygame.display.update()
        self.FramePerSec.tick(self.FPS)

    def close(self):
        pass


pygame 2.5.1 (SDL 2.28.2, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


## TEST ENVIRONMENT WORKING

In [2]:
env = droneEnv(True, False)

obs = env.reset()
env.render("yes")

print("Observation space:")
print(env.observation_space)
print("")
print("Action space:")
print(env.action_space)
print("")
print("Action space sample:")
print(env.action_space.sample())

# Choose an action to execute n_steps times
action = np.array([0.0, 1.0])
n_steps = 10
for step in range(n_steps):
    print("Step {}".format(step + 1))
    obs, reward, done, info = env.step(action)
    print("obs=", obs, "reward=", reward, "done=", done)
    env.render("yes")
    if done:
        print("Done!", "reward=", reward)
        break


Observation space:
Box([-inf -inf -inf -inf -inf -inf], [inf inf inf inf inf inf], (6,), float32)

Action space:
Box([-1. -1.], [1. 1.], (2,), float32)

Action space sample:
[0.10401974 0.22499242]
Step 1
obs= [-0.03926991  0.00418868 -0.75        0.36703703  1.214614    1.2050583 ] reward= -0.06960023666881161 done= False
Step 2
obs= [-0.14398967  0.03454228 -1.5         0.3669645   1.2151002   1.1760951 ] reward= -0.06958503453605186 done= False
Step 3
obs= [-0.31415927  0.1170206  -2.25        0.3666386   1.2170571   1.1289977 ] reward= -0.06950037154027486 done= False
Step 4
obs= [-0.5497787   0.27655566 -3.          0.3656828   1.2220284   1.0654168 ] reward= -0.06922368461317027 done= False
Step 5
obs= [-0.850848    0.53502405 -3.75        0.3634504   1.231879    0.98749965] reward= -0.06853309900568291 done= False
Step 6
obs= [-1.2173672   0.907548   -4.5         0.358968    1.2484827   0.89771855] reward= -0.06707847123355268 done= False
Step 7
obs= [-1.6493361   1.3969848  -5.

## TRAIN TD3 AGENT

In [6]:
from stable_baselines3 import TD3
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import CheckpointCallback
import wandb
from wandb.integration.sb3 import WandbCallback


run = wandb.init(
    project="CSP",
    sync_tensorboard=True,
    monitor_gym=True,
)

# Create log dir
log_dir = "tmp/"
os.makedirs(log_dir, exist_ok=True)

# Create and wrap the environment
env = droneEnv(False, False)
env = Monitor(env, log_dir)

# Create TD agent
model = TD3("MlpPolicy", env, verbose=1, tensorboard_log=log_dir)



training_steps = 1_000_000

# Create checkpoint callback
checkpoint_callback = CheckpointCallback(
    save_freq=training_steps, save_path=log_dir, name_prefix="td3_model_v5"
)

# Train the agent
model.learn(
    total_timesteps=training_steps,
    callback=[
        checkpoint_callback,
        WandbCallback(
            gradient_save_freq=100,
            model_save_path=f"models/{run.id}",
            model_save_freq=training_steps,
            verbose=2,
        ),
    ],
)

wandb: WARNING When using several event log directories, please call `wandb.tensorboard.patch(root_logdir="...")` before `wandb.init`


Using cpu device
Wrapping the env in a DummyVecEnv.
Logging to tmp/TD3_4
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 38       |
|    ep_rew_mean     | -958     |
| time/              |          |
|    episodes        | 4        |
|    fps             | 152      |
|    time_elapsed    | 0        |
|    total_timesteps | 152      |
| train/             |          |
|    actor_loss      | 20.8     |
|    critic_loss     | 5.12e+04 |
|    learning_rate   | 0.001    |
|    n_updates       | 34       |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 40.4     |
|    ep_rew_mean     | -983     |
| time/              |          |
|    episodes        | 8        |
|    fps             | 32       |
|    time_elapsed    | 9        |
|    total_timesteps | 323      |
| train/             |          |
|    actor_loss      | 31.1     |
|    critic_loss     | 3.67e+04 |
|    lear

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 37.7     |
|    ep_rew_mean     | -991     |
| time/              |          |
|    episodes        | 68       |
|    fps             | 36       |
|    time_elapsed    | 69       |
|    total_timesteps | 2562     |
| train/             |          |
|    actor_loss      | 197      |
|    critic_loss     | 1.51e+03 |
|    learning_rate   | 0.001    |
|    n_updates       | 2453     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 37.6     |
|    ep_rew_mean     | -991     |
| time/              |          |
|    episodes        | 72       |
|    fps             | 36       |
|    time_elapsed    | 73       |
|    total_timesteps | 2709     |
| train/             |          |
|    actor_loss      | 210      |
|    critic_loss     | 1.5e+03  |
|    learning_rate   | 0.001    |
|    n_updates       | 2599     |
--------------

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 37.2     |
|    ep_rew_mean     | -985     |
| time/              |          |
|    episodes        | 136      |
|    fps             | 40       |
|    time_elapsed    | 124      |
|    total_timesteps | 5094     |
| train/             |          |
|    actor_loss      | 369      |
|    critic_loss     | 645      |
|    learning_rate   | 0.001    |
|    n_updates       | 4987     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 37.3     |
|    ep_rew_mean     | -986     |
| time/              |          |
|    episodes        | 140      |
|    fps             | 40       |
|    time_elapsed    | 128      |
|    total_timesteps | 5248     |
| train/             |          |
|    actor_loss      | 342      |
|    critic_loss     | 620      |
|    learning_rate   | 0.001    |
|    n_updates       | 5135     |
--------------

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 37.4     |
|    ep_rew_mean     | -988     |
| time/              |          |
|    episodes        | 204      |
|    fps             | 42       |
|    time_elapsed    | 178      |
|    total_timesteps | 7641     |
| train/             |          |
|    actor_loss      | 468      |
|    critic_loss     | 228      |
|    learning_rate   | 0.001    |
|    n_updates       | 7531     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 37.4     |
|    ep_rew_mean     | -989     |
| time/              |          |
|    episodes        | 208      |
|    fps             | 42       |
|    time_elapsed    | 181      |
|    total_timesteps | 7788     |
| train/             |          |
|    actor_loss      | 484      |
|    critic_loss     | 158      |
|    learning_rate   | 0.001    |
|    n_updates       | 7681     |
--------------

...

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 240      |
|    ep_rew_mean     | 805      |
| time/              |          |
|    episodes        | 5372     |
|    fps             | 48       |
|    time_elapsed    | 19403    |
|    total_timesteps | 941643   |
| train/             |          |
|    actor_loss      | -144     |
|    critic_loss     | 394      |
|    learning_rate   | 0.001    |
|    n_updates       | 941330   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 240      |
|    ep_rew_mean     | 806      |
| time/              |          |
|    episodes        | 5376     |
|    fps             | 48       |
|    time_elapsed    | 19422    |
|    total_timesteps | 942603   |
| train/             |          |
|    actor_loss      | -142     |
|    critic_loss     | 463      |
|    learning_rate   | 0.001    |
|    n_updates       | 942290   |
--------------

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 240      |
|    ep_rew_mean     | 822      |
| time/              |          |
|    episodes        | 5440     |
|    fps             | 48       |
|    time_elapsed    | 19741    |
|    total_timesteps | 957963   |
| train/             |          |
|    actor_loss      | -149     |
|    critic_loss     | 481      |
|    learning_rate   | 0.001    |
|    n_updates       | 957650   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 240      |
|    ep_rew_mean     | 824      |
| time/              |          |
|    episodes        | 5444     |
|    fps             | 48       |
|    time_elapsed    | 19761    |
|    total_timesteps | 958923   |
| train/             |          |
|    actor_loss      | -149     |
|    critic_loss     | 378      |
|    learning_rate   | 0.001    |
|    n_updates       | 958610   |
--------------

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 239      |
|    ep_rew_mean     | 807      |
| time/              |          |
|    episodes        | 5508     |
|    fps             | 48       |
|    time_elapsed    | 20087    |
|    total_timesteps | 974179   |
| train/             |          |
|    actor_loss      | -153     |
|    critic_loss     | 469      |
|    learning_rate   | 0.001    |
|    n_updates       | 973866   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 239      |
|    ep_rew_mean     | 807      |
| time/              |          |
|    episodes        | 5512     |
|    fps             | 48       |
|    time_elapsed    | 20109    |
|    total_timesteps | 975139   |
| train/             |          |
|    actor_loss      | -151     |
|    critic_loss     | 456      |
|    learning_rate   | 0.001    |
|    n_updates       | 974826   |
--------------

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 239      |
|    ep_rew_mean     | 808      |
| time/              |          |
|    episodes        | 5576     |
|    fps             | 48       |
|    time_elapsed    | 20451    |
|    total_timesteps | 990499   |
| train/             |          |
|    actor_loss      | -159     |
|    critic_loss     | 531      |
|    learning_rate   | 0.001    |
|    n_updates       | 990186   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 240      |
|    ep_rew_mean     | 816      |
| time/              |          |
|    episodes        | 5580     |
|    fps             | 48       |
|    time_elapsed    | 20472    |
|    total_timesteps | 991459   |
| train/             |          |
|    actor_loss      | -154     |
|    critic_loss     | 466      |
|    learning_rate   | 0.001    |
|    n_updates       | 991146   |
--------------

OSError: [WinError 1314] A required privilege is not held by the client: 'C:\\Users\\AVISH\\models\\cy5qnoel\\model.zip' -> 'C:\\Users\\AVISH\\wandb\\run-20231210_055221-cy5qnoel\\files\\model.zip'

## TEST TRAINED AGENT IN THE ENVIRONMENT

In [ ]:
import torch as th
from stable_baselines3 import TD3
MODEL_PATH =r"C:\Users\AVISH\tmp\td_model_v1_1000000_steps.zip"

# Create and wrap the environment
env = droneEnv(True, False)

# Load the trained agent
model = TD3.load(MODEL_PATH, env=env)

# Evaluate the agent
for i in range(5):
    obs = env.reset()
    done = False
    episode_reward = 0
    while not done:
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, done, info = env.step(action)
        episode_reward += reward
    print("Episode reward", episode_reward)
    env.render("yes")